# SageMakerCV PyTorch Tutorial

SageMakerCV is a collection of computer vision tools developed to take full advantage of Amazon SageMaker by providing state of the art model accuracy, training speed, and training cost reductions. SageMakerCV is based on the lessons we learned from developing the record breaking computer vision models we announced at Re:Invent in 2019 and 2020, along with talking to our customers and understanding the challenges they faced in training their own computer vision models.

The tutorial in this notebook walks through using SageMakerCV to train Mask RCNN on the COCO dataset. The only prerequisite is to setup SageMaker studio, the instructions for which can be found in [Onboard to Amazon SageMaker Studio Using Quick Start](https://docs.aws.amazon.com/sagemaker/latest/dg/onboard-quick-start.html). Everything else, from getting the COCO data to launching a distributed training cluster, is included here.

## Setup and Roadmap

Before diving into the tutorial itself, let's take a minute to discuss the various tools we'll be using.

#### SageMaker Studio
[SageMaker Studio](https://aws.amazon.com/sagemaker/studio/) is a machine learning focused IDE where you can interactively develop models and launch SageMaker training jobs all in one place. SageMaker Studio provides a Jupyter Lab like environment, but with a number of enhancements. We'll just scratch the surface here. See the [SageMaker Studio Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html) for more details.

For our purposes, the biggest difference from regular Jupyter Lab is that SageMaker Studio allows you to change your compute resources as needed, by connecting notebooks to Docker containers on different ML instances. This is a little confusing to just describe, so let's walk through an example.

Once you've completed the setup on [Onboard to Amazon SageMaker Studio Using Quick Start](https://docs.aws.amazon.com/sagemaker/latest/dg/onboard-quick-start.html), go to the [SageMaker Console](https://us-west-2.console.aws.amazon.com/sagemaker) and click `Open SageMaker Studio` near the top right of the page.

<img src="../assets/SageMaker_console.png" style="width: 600px">

If you haven't yet created a user, do so via the link at the top left of the page. Give it any name you like. For execution role, you can either use an existing SageMaker role, or create a new one. If you're unsure, create a new role. On the `Create IAM Role` window, make sure to select `Any S3 Bucket`. 

<img src="../assets/Create_IAM_role.png" style="width: 600px">

Back on the SageMaker Studio page, select `Open Studio` next to the user you just created.

<img src="../assets/Studio_domain.png" style="width: 600px">

This will take a couple minutes to start up the first time. Once it starts, you'll have a Jupyter Lab like interface running on a small instance with an attached EBS volume. Let's start by taking a look at the `Launcher` tab.

<img src="../assets/Studio_launcher.png" style="width: 750px">

If you don't see the `Launcher`, you can bring one up by clicking the `+` on the menu bar in the upper left corner.

<img src="../assets/Studio_menu_bar.png" style="width: 600px">

The `Launcher` gives you access to all kinds of tools. This is where you can create new notebooks, text files, or get a terminal for your instance. Try the `System Terminal`. This gives you a new terminal tab for your Studio instance. It's useful for things like downloading data or cloning github repos into studio. For example, you can run `aws s3 ls` to browse your current S3 buckets. Go ahead and clone this repo onto Studio with 

`git clone https://github.com/aws-samples/amazon-sagemaker-cv`

Let's look at the launcher one more time. Bring another one up with the `+`. Notice you have an option for `Select a SageMaker image` above the button to launch a notebook. This allows you to select a Docker image that will launch on a new instance. The notebook you create will be attached to that new instance, along with the EBS volume on your Studio instance. Let's try it out. On the `Launcher` page, click the drop down menu next to `Select a SageMaker Image` and select `PyTorch 1.6 Python 3.6 (Optimzed for GPU)`, then click the `Notebook` button below the dropdown.

<img src="../assets/Select_pytorch_image.png" style="width: 600px">

Take a look at the upper righthand corner of the notebook. 

<img src="../assets/notebook_pytorch_kernel.png" style="width: 600px">

The `Ptyhon 3 (PyTorch 1.6 Python 3.6 GPU Optimized)` refers to the kernel associated with this notebook. The `Unknown` refers to the current instance type. Click `Unknown` and select `ml.g4dn.xlarge`.

<img src="../assets/instance_types.png" style="width: 600px">

This will launch a `ml.g4dn.xlarge` instance and attach this notebook to it. This will take a couple of minutes, because Studio needs to download the PyTorch Docker image to the new instance. Once an instance has started, launching new notebooks with the same instance type and kernel is immediate. You'll also see the `Unknown` replaced with and instance description `4 vCPU + 16 GiB + 1 GPU`. You can also change instance as needed. Say you want to run your notebook on a `ml.p3dn.24xlarge` to get 8 GPUs. To change instances, just click the instance description. To get more instances in the menu, deselect `Fast launch only`.

Once your notebook is up and running, you can also get a terminal into your new instance.

<img src="../assets/Launch_terminal.png" style="width: 600px">

This can be useful for customizing your image with setup scripts, pip installing new packages, or using mpi to launch multi GPU training jobs. Click to get a terminal and run `ls`. Note that you have the same directories as your main Studio instance. Studio will attach the same EBS volume to all the instances you start, so all your files and data are shared across any notebooks you start. This means that you can prototype a model on a single GPU instance, then switch to a multi GPU instance while still having access to all of your data and scripts.

Finally, when you want to shut down instances, click the circle with a square in it on the left hand side.

<img src="../assets/running_instances.png" style="width: 600px">

This shows your current running instances, and the Docker containers attached to those instances. To shut them down, just click the power button to their right.

Now that we've explored studio a bit, let's get started with SageMakerCV. If you followed the instructions above to clone the repo, you should have `amazon-sagemaker-cv` in the file browser on the left. Navigate to `amazon-sagemaker-cv/pytorch/tutorial.ipynb` to open this notebook on your instance. If you still have a `g4dn` running, it should automatically attach to it.

The rest of this notebook is broken into 4 sections.

- Installing SageMakerCV and Downloading the COCO Data

Since we're using the base AWS Deep Learning Container image, we need to add the SageMakerCV tools. Then we'll download the COCO dataset and upload it to S3.

- Prototyping in Studio

We'll walk through how to train a model on Studio, how SageMakerCV is structured, and how you can add your own models and features.

- Launching a SageMaker Training Job

There's lots of bells and whistles available to train your models fast, an on large datasets. We'll put a lot of those together to launch a high performance training job. Specifically, we'll create a training job with 4 P4d.24xlarge instances connected with 400 GB EFA, and streaming our training data from S3, so we don't have to load the dataset onto the instances before training. You could even use this same configuration to train on a dataset that wouldn't fit on the instances. If you'd rather only launch a smaller (or larger) training cluster, we'll discuss how to modify configuration.

- Testing Our Model

Finally, we'll take the output trained Mask RCNN model and visualize its performance in Studio.

#### Installing SageMakerCV

To install SageMakerCV on the PyTorch Studio Docker, just run `pip install -e .` in the `amazon-sagemaker-cv/pytorch` directory. You can do this with either an image terminal, or by running the paragraph below. Note that we use the `-e` option. This will keep the SageMakerCV modules editable, so any changes you make will be launched on your training job.

In [1]:
!pip install -e .

Obtaining file:///root/blog_post/amazon-sagemaker-cv/pytorch
  Using cached https://sagemakercv.s3.us-west-2.amazonaws.com/cocoapi/pycocotools-2.0%2Bnv0.6.0-cp38-cp38-linux_x86_64.whl (2.0 MB)
  Using cached https://sagemakercv.s3.us-west-2.amazonaws.com/utils/pt-1.9/smcv_utils-0.0.1-cp38-cp38-linux_x86_64.whl (11.1 MB)
  Using cached https://aws-s3-plugin.s3.us-west-2.amazonaws.com/binaries/0.0.1/1c3e69e/awsio-0.0.1-cp38-cp38-manylinux1_x86_64.whl (3.1 MB)
  Attempting uninstall: sagemakercv
    Found existing installation: sagemakercv 0.1
    Uninstalling sagemakercv-0.1:
      Successfully uninstalled sagemakercv-0.1
  Running setup.py develop for sagemakercv


***
### Setup on S3 and Download COCO data

Next we need to setup an S3 bucket for all our data and results. Enter a name for your S3 bucket below. You can either create a new bucket, or use an existing bucket. If you use an existing bucket, make sure it's in the same region where you plan to run training. For new buckets, we'll specify that it needs to be in the current SageMaker region. By default we'll put everything in an S3 location on your bucket named `smcv-tutorial`, and locally in `/root/smcv-tutorial`, but you can change these locations. 

In [2]:
S3_BUCKET = 'sagemaker-smcv-tutorial' # Don't include s3:// in your bucket name
S3_DIR = 'smcv-pytorch-tutorial'
LOCAL_DATA_DIR = '/root/smcv-pytorch-tutorial' #for reasons detailed in Destributed Training, do not put this dir in your source dir

In [3]:
import os
import zipfile
from pathlib import Path
from s3fs import S3FileSystem
from concurrent.futures import ThreadPoolExecutor
import boto3
from botocore.client import ClientError
from tqdm import tqdm

In [4]:
s3 = boto3.resource('s3')
boto_session = boto3.session.Session()
region = boto_session.region_name

# Check if bucket exists. If it doesn't, create it.

try:
    bucket = s3.meta.client.head_bucket(Bucket=S3_BUCKET)
    print(f"S3 Bucket {S3_BUCKET} Exists")
except ClientError:
    print(f"Creating Bucket {S3_BUCKET}")
    bucket = s3.create_bucket(Bucket=S3_BUCKET, CreateBucketConfiguration={'LocationConstraint': region})

S3 Bucket sagemaker-smcv-tutorial Exists


***

Next we'll download the COCO data to Studio, unzip the files, and upload to S3. The reason we want the data in two places is that it's convenient to have the data locally on Studio for prototyping. We also want to unarchive the data before moving it to S3 so that we can stream it to our training instances instead of downloading it all at once.

Once this is finished, you'll have copies of the COCO data on your Studio instance, and in S3. Be careful not to open the `data/coco/train2017` dir in the Studio file browser. It contains 118287 images, and can cause your web browser to crash. If you need to browse these files, use the terminal.

This only needs to be done once, and only if you don't already have the data. The COCO 2017 dataset is about 20GB, so this step takes around 30 minutes to complete. The next paragraph sets up all the file directories we'll use for downloading, and later in training. 

In [5]:
COCO_URL="http://images.cocodataset.org"
ANNOTATIONS_ZIP="annotations_trainval2017.zip"
TRAIN_ZIP="train2017.zip"
VAL_ZIP="val2017.zip"
COCO_DIR=os.path.join(LOCAL_DATA_DIR, 'data', 'coco')
os.makedirs(COCO_DIR, exist_ok=True)
S3_DATA_LOCATION=os.path.join("s3://", S3_BUCKET, S3_DIR, "data", "coco")
S3_WEIGHTS_LOCATION=os.path.join("s3://", S3_BUCKET, S3_DIR, "data", "weights")
WEIGHTS_DIR=os.path.join(LOCAL_DATA_DIR, 'data', 'weights')
os.makedirs(WEIGHTS_DIR, exist_ok=True)
R50_WEIGHTS="resnet50.pkl"
R50_WEIGHTS_SRC="https://sagemakercv.s3.us-west-2.amazonaws.com/weights/pytorch"

***

And this paragraph will download everything, and take around 30 minutes to complete.

In [7]:
print("Downloading annotations")
!wget -O $COCO_DIR/$ANNOTATIONS_ZIP $COCO_URL/annotations/$ANNOTATIONS_ZIP
!unzip $COCO_DIR/$ANNOTATIONS_ZIP -d $COCO_DIR
!rm $COCO_DIR/$ANNOTATIONS_ZIP
!aws s3 cp --recursive $COCO_DIR/annotations $S3_DATA_LOCATION/annotations

print("Downloading COCO training data")
!wget -O $COCO_DIR/$TRAIN_ZIP $COCO_URL/zips/$TRAIN_ZIP

# train data has ~128000 images. Unzip is too slow, about 1.5 hours beceause of disk read and write speed on the EBS volume. 
# This technique is much faster because it grabs all the zip metadata at once, then uses threading to unzip multiple files at once.
print("Unzipping COCO training data")
train_zip = zipfile.ZipFile(os.path.join(COCO_DIR, TRAIN_ZIP))
jpeg_files = [image.filename for image in train_zip.filelist if image.filename.endswith('.jpg')]
os.makedirs(os.path.join(COCO_DIR, 'train2017'))
with ThreadPoolExecutor() as executor:
    threads = list(tqdm(executor.map(lambda x: train_zip.extract(x, COCO_DIR), jpeg_files), total=len(jpeg_files)))

# same issue for uploading to S3. this uploads in parallel, and is faster than using aws cli in this case.
print("Uploading COCO training data to S3")
train_images = [i for i in Path(os.path.join(COCO_DIR, 'train2017')).glob("*.jpg")]
s3fs = S3FileSystem()
with ThreadPoolExecutor() as executor:
    threads = list(tqdm(executor.map(lambda image: s3fs.put(image.as_posix(), os.path.join(S3_DATA_LOCATION, 'train2017', image.name)), 
                                     train_images), total=len(train_images)))
# !rm $COCO_DIR/$TRAIN_ZIP

print("Downloading COCO validation data")
!wget -O $COCO_DIR/$VAL_ZIP $COCO_URL/zips/$VAL_ZIP
# switch to also threading
!unzip -q $COCO_DIR/$VAL_ZIP -d $COCO_DIR
val_images = [i for i in Path(os.path.join(COCO_DIR, 'val2017')).glob("*.jpg")]
with ThreadPoolExecutor() as executor:
    threads = list(tqdm(executor.map(lambda image: s3fs.put(image.as_posix(), os.path.join(S3_DATA_LOCATION, 'val2017', image.name)), 
                                     val_images), total=len(val_images)))
# !rm $COCO_DIR/$VAL_ZIP

# grab resnet backbone from public S3 bucket
!wget -O $WEIGHTS_DIR/$R50_WEIGHTS $R50_WEIGHTS_SRC/$R50_WEIGHTS
!aws s3 cp $WEIGHTS_DIR/$R50_WEIGHTS $S3_WEIGHTS_LOCATION/$R50_WEIGHTS

print("FINISHED!")

--2021-11-12 23:15:13--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.224.120
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.224.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/root/smcv-pytorch-tutorial/data/coco/annotations_trainval2017.zip’

/root/smcv-pytorch-  14%[=>                  ]  34.20M  15.6MB/s               ^C
Archive:  /root/smcv-pytorch-tutorial/data/coco/annotations_trainval2017.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /root/smcv-pytorch-tutorial/data/coco/annotations_trainval2017.zip or
        /root/smcv-pytorch-tut

KeyboardInterrupt: 

***
### Training on Studio

Now that we have the data, we can get to training a Mask RCNN model to detect objects in the COCO dataset images. 

Since training on a single GPU can take days, we'll just train for a couple thousands steps, and run a single evaluation to make sure our model is at least starting to learn something. We'll train a full model on a larger cluster of GPUs in a SageMaker training job.

The reason we first want to train in Studio is that we want to dig a bit into the SageMakerCV framework, and talk about the model architecture, since we expect many users will want to modify models for their own use cases.

#### Mask RCNN

First, just a very brief overview of Mask RCNN. If you would like a more in depth examination, we recommend taking a look at the [original paper](https://arxiv.org/abs/1703.06870), the [feature pyramid paper](https://arxiv.org/abs/1612.03144) which describes a popular architectural change we'll use in our model, and blog posts from [viso.ai](https://viso.ai/deep-learning/mask-r-cnn/), [tryo labs](https://tryolabs.com/blog/2018/01/18/faster-r-cnn-down-the-rabbit-hole-of-modern-object-detection/), [Jonathan Hui](https://jonathan-hui.medium.com/image-segmentation-with-mask-r-cnn-ebe6d793272), and [Lilian Weng](https://lilianweng.github.io/lil-log/2017/12/31/object-recognition-for-dummies-part-3.html).

Mask RCNN is a two stage object detection model that locates objects in images by places bounding boxes around, and segmentation masks over, any object for which the model is trained to find. It also provides classifcations for each object.

<img src="../assets/traffic.png" style="width: 1200px">

Mask RCNN is called a two stage model because it performs detection in two steps. The first identified any objects in the image, versus background. The second stage determines the specific class of each object, and applies the segmentation mask. Below is an architectural diagram of the model. Let's walk through each step.

<img src="../assets/mask_rcnn_arch.jpeg" style="width: 1200px">
Credit: Jonathan Hui

The `Convolution Network` is often referred to as the model backbone. This is a pretrained image classification model, commonly ResNet, which has been trained on a large image classification dataset, like ImageNet. The classification layer is removed, and instead the backbone outputs a set of convolution feature maps. The idea is, the classification model learned to identify objects in the process of classifying images, and now we can use that information to build a more complex model that can find those objects in the image. We want to pretrain because training the backbone at the same time as training the object detector tends to be very unstable.

One additional component that is sometimes added to the backbone is a `Fearure Pyramid Network`. This take the outputs of the backbone, and combines them to together into a new set of feature maps by perform both up and down convolutions. The idea is that the different sized feature maps will help the model detect images of different sizes. The feature pyramid also helps with this, by allowing the different feature maps to share information with each other.

The outputs of the feature pyramid are then passed to the `Region Proposal Network` which is responsible for finding regions of the image that might contain an object (this is the first of the two stages). The RPN will output several hundred thousand regions, each with a probability of containing an object. We'll typically take the top few thousand most likely regions. Because these several thousand regions will usually have a lot of overlap, we perform [non-max supression](https://towardsdatascience.com/non-maximum-suppression-nms-93ce178e177c), which removed regions with large areas of overlap. This gives us a set of `regions of interest` regions of the image that we think might contain an image.

Next, we use those regions to crop out the corresponding sections of the feature maps that came from the feature pyramid network using a technique called [ROI align](https://firiuza.medium.com/roi-pooling-vs-roi-align-65293ab741db).

We pass our cropped feature maps to the `box head` which classifies each region into either a specific object category, or as background. It also refines the position of the bounding box. In Mask RCNN, we also pass the feature maps to a `mask head` which produces a segmentation mask over the object.

#### SageMakerCV Internals

An important feature of Mask RCNN is its multiple heads. One head constructs a bounding box, while another creates a mask. These are referred to as the `ROI heads`. It's common for users to extend this and other two stage models by adding their own ROI heads. For example, a keypoint head it common. Doing so means modifying SageMakerCV's internals, so let's talk about those for a second. 

The high level Mask RCNN model can be found in `amazon-sageamaker-cv/pytorch/sagemakercv/detection/detector/generatlized_rcnn.py`. If you trace through the forward function, you'll see that the model first passes an image through the backbone (which also contains the feature pyramid), then the RPN in the graphable module. Then results are then passed through non-max suppression, and into the roi heads. 

Probably the most important feature to be aware of are the `build` imports at the top. Each section of the model has an associated build function `(build_backbone, build_rpn, build_roi_heads)`. These functions simplify building the model by letting us pass in a single configuration file for building all the different pieces.  

For example, if you open `amazon-sageamaker-cv/pytorch/sagemakercv/detection/roi_heads/roi_heads.py`, you'll find the `build_roi_heads` function at the bottom. To add a new head, you would write a torch module with its own build function, and call the build function from here.

For example, say you want to add a keypoint head to the model. An example keypoint module and associate build function is in `amazon-sageamaker-cv/pytorch/sagemakercv/detection/roi_heads/keypoint_head/keypoint_head.py`. To enable the keypoint head, you would set `cfg.MODEL.KEYPOINY_ON=True` and add the keypoint parameters to your configuration yaml file.

SageMakerCV uses similar build functions for the optimizers and schedulers, which you can add to or modify in the `amazon-sageamaker-cv/pytorch/sagemakercv/training/optimizers/` directory. 

Finally, data loading tools are located in `amazon-sagemaker-cv/pytorch/data/`. Here you can add a new dataset, sampler, and preprocessing data transformations. Data loaders are constructed in the `build.py` file. Notice the `@DATASETS.register("COCO")` decorator at the top of the COCO `make_coco_dataloader` function. This adds the function to a dictionary of datasets, so that when you specify `COCO` in yout configuration file, the `make_data_loader` knows which data loader to create.

#### Setting Up Training

Let's actually use some of these functions to train a model.

Start by importing the default configuration file.

In [6]:
from configs import cfg

***
We use the [yacs](https://github.com/rbgirshick/yacs) format for configuration files. If you want to see the entire config, run `print(cfg.dump())` but this prints out a lot, and to not overwhelm you with too much information, we'll just focus on the bits we want to change for this model.

***
First, let's put in all the file directories for the data and weights we downloaded in the previous section, as well as an output directory for the model results.

In [7]:
cfg.INPUT.TRAIN_INPUT_DIR = os.path.join(COCO_DIR, "train2017")
cfg.INPUT.VAL_INPUT_DIR = os.path.join(COCO_DIR, "val2017")
cfg.INPUT.TRAIN_ANNO_DIR = os.path.join(COCO_DIR, "annotations", "instances_train2017.json")
cfg.INPUT.VAL_ANNO_DIR = os.path.join(COCO_DIR, "annotations", "instances_val2017.json")
cfg.OUTPUT_DIR = os.path.join(LOCAL_DATA_DIR, "model-output")
# create output dir if it doesn't exist
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# backbone weights file
cfg.MODEL.WEIGHT=os.path.join(WEIGHTS_DIR, R50_WEIGHTS)

****

Next we need to setup our data loader. The data loader is a tool in PyTorch that handles loading and applying transformations on our dataset. For more information see the [PyTorch Dataset and Data Loader Documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

For our purposes, we need to set two parameters. The size divisibility has to do with downsampling that occurs in the backbone and feature pyramid. The feature maps generated by these layers must evenly divide into the original image size. For example, if the input image is 1344x1344, the smallest feature map will be 42x42, which means the input image must be divisible by 32 `(42x32=1344)`. The `SIZE_DIVISIBLITY` parameter makes sure all images are resized to be multiples of 32. 

The number of workers has to do with the number of background processes that will run the data loader in parallel. This can be useful for really high performance systems, to make sure our data loader is feeding data to the GPUs fast enough. However, for prototyping, these background processes can cause memory problems, so we'll turn it off when running in a notebook. We'll switch them back on later for our larger training job.

In [8]:
# dataloader settings
cfg.DATALOADER.SIZE_DIVISIBILITY=32
cfg.DATALOADER.NUM_WORKERS=0

***
This section specifies model details, including the type of model, and internal hyperparameters. We wont cover the details of all of these, but more information can be found in this blog posts listed above, as well as the original paper.

In [9]:
cfg.MODEL.META_ARCHITECTURE="GeneralizedRCNN" # The type of model we're training. found in amazon-sagemaker-cv/pytorch/sagemakercv/detection/detector/generalized_rcnn.py
cfg.MODEL.RESNETS.TRANS_FUNC="BottleneckWithFixedBatchNorm" # Type of bottleneck function in the Resnet50 backbone. see https://arxiv.org/abs/1512.03385
cfg.MODEL.BACKBONE.CONV_BODY="R-50-FPN" # Type of backbone, Resnet50 with feature pyramid network
cfg.MODEL.BACKBONE.OUT_CHANNELS=256 # number of channels on the output feature maps from the backbone
cfg.MODEL.RPN.USE_FPN=True # Use Feature Pyramid. RPN needs to know this since FPN adds an extra feature map
cfg.MODEL.RPN.ANCHOR_STRIDE=(4, 8, 16, 32, 64) # positions of anchors, see blog posts for details
cfg.MODEL.RPN.PRE_NMS_TOP_N_TRAIN=2000 # top N anchors to keep before non-max suppression during training
cfg.MODEL.RPN.PRE_NMS_TOP_N_TEST=1000 # top N anchors to keep before non-max suppression during testing
cfg.MODEL.RPN.POST_NMS_TOP_N_TEST=1000 # top N anchors to keep after non-max suppression during testing
cfg.MODEL.RPN.FPN_POST_NMS_TOP_N_TRAIN=1000 # top N anchors to keep after non-max suppression during training
cfg.MODEL.RPN.FPN_POST_NMS_TOP_N_TEST=1000 # top N anchors to keep before non-max suppression during training
cfg.MODEL.RPN.FPN_POST_NMS_TOP_N_PER_IMAGE=True # Run NMS per FPN level
cfg.MODEL.RPN.LS=0.1 # label smoothing improves performance on less common categories

# ROI Heads
cfg.MODEL.ROI_HEADS.USE_FPN=True # Use Feature Pyramid. ROI needs to know this since FPN adds an extra feature map
cfg.MODEL.ROI_HEADS.BBOX_REG_WEIGHTS=(10., 10., 5., 5.) # Regression wieghts for bounding boxes, see blog posts
cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION=7 # Pixel size of region cropped from feature map
cfg.MODEL.ROI_BOX_HEAD.POOLER_SCALES=(0.25, 0.125, 0.0625, 0.03125) # Pooling for ROI align
cfg.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO=2 # Sampling for ROI Align
cfg.MODEL.ROI_BOX_HEAD.FEATURE_EXTRACTOR="FPN2MLPFeatureExtractor" # Type of ROI feature extractor found in SageMakerCV core utils
cfg.MODEL.ROI_BOX_HEAD.PREDICTOR="FPNPredictor" # Predictor type used for inference found in SageMakerCV core utils
cfg.MODEL.ROI_BOX_HEAD.LOSS="GIoULoss" # Use GIoU loss, improves box performance https://giou.stanford.edu/GIoU.pdf
cfg.MODEL.ROI_BOX_HEAD.DECODE=True # Convert boxes to pixel positions
cfg.MODEL.ROI_BOX_HEAD.CARL=True # Use carl loss https://arxiv.org/pdf/1904.04821.pdf
cfg.MODEL.ROI_MASK_HEAD.POOLER_SCALES=(0.25, 0.125, 0.0625, 0.03125) # Mask head ROI align
cfg.MODEL.ROI_MASK_HEAD.FEATURE_EXTRACTOR="MaskRCNNFPNFeatureExtractor" # Mask feature extractor type in SageMakerCV core utils
cfg.MODEL.ROI_MASK_HEAD.PREDICTOR="MaskRCNNC4Predictor" # Predictor used for inference
cfg.MODEL.ROI_MASK_HEAD.POOLER_RESOLUTION=14 # Pixel size of region cropped from feature map
cfg.MODEL.ROI_MASK_HEAD.POOLER_SAMPLING_RATIO=2 # ROI align sampling ratio
cfg.MODEL.ROI_MASK_HEAD.RESOLUTION=28 # output resolution of mask
cfg.MODEL.ROI_MASK_HEAD.SHARE_BOX_FEATURE_EXTRACTOR=False # share feature extractor between box and mask heads
cfg.MODEL.MASK_ON=True # use mask head

***
Next we set up the configuration for training, including the optimizer, hyperparameters, batch size, and training length. Batch size is global, so if you set a batch size of 64 across 8 GPUs, it will be a batch size of 8 per GPU. SageMakerCV currently supports the following optimizere: SGD (stochastic gradient descent), Adam, Lamb, and NovoGrad [link - this speeds up training by allowing increased batch sizes], and the following learning rate schedulers: stepwise and cosine decay. New, custom optimizers and schedulers can be added by modifying the `sagemakercv/training/build.py` file.

For training on Studio, we'll just run for a few hundred steps. We'll be using SageMaker training instances for the full training on multiple GPUs.

We also set the mixed precision optimization level. This is a value between O0-O4. O0 means no optimization, and training purely in FP32. O1-O3 are varying degrees of mixed precision, explained in the [Nvidia Apex documentation](https://nvidia.github.io/apex/amp.html). O4 is a special optimization level that combines pure FP16 training with channel last memory optimizations. This is the optimization we used to achieve the performance we announced at Re:Invent in 2020. However, optimization levels O2-O4 introduce some numerical instability, and can take a lot of tuning to train properly. For most users, we recommend starting on O1, since this provides most performance benefits of mixed precision, while retaining good numeric stability.

In [10]:
cfg.SOLVER.OPTIMIZER="NovoGrad" # Type of optimizer, NovoGrad, Adam, SGD, Lamb
cfg.SOLVER.BASE_LR=0.004 # Learning rate after warmup [Suggested values]
#cfg.SOLVER.BASE_MOMENTUM=0.9 #Momentum value for SGD
cfg.SOLVER.BETA1=0.9 # Beta value for Novograd, Adam, and Lamb
cfg.SOLVER.BETA2=0.4 # Beta value for Novograd, Adam, and Lamb
cfg.SOLVER.ALPHA=.1 # Alpha for final value of cosine decay
cfg.SOLVER.LR_SCHEDULE="COSINE" # Decay type, COSINE or MULTISTEP
cfg.SOLVER.IMS_PER_BATCH=8 # Global training batch size, must be a multiple of the number of GPUs
cfg.SOLVER.WEIGHT_DECAY=0.0005 # Training weight decay applied as decoupled weight decay on optimizer. [Paper link]
cfg.SOLVER.MAX_ITER=100 # Total number of training steps for local training
cfg.SOLVER.WARMUP_FACTOR=.01 # Starting learning rate as a multiple of the BASE_LR
cfg.SOLVER.WARMUP_ITERS=100 # Number of warmup steps to reach BASE_LR
cfg.SOLVER.GRADIENT_CLIPPING=0.0 # Gradient clipping norm, leave as 0.0 to disable gradient clipping
cfg.OPT_LEVEL="O4" # Mixed precision optimization level
cfg.TEST.IMS_PER_BATCH=1 # Evaluation batch size, must be a multiple of the number of GPUs
cfg.TEST.PER_EPOCH_EVAL=True # Eval after every epoch or only at the end of training for local training

***
Finally, SageMakerCV includes a number of training hooks, ie. tools that will trigger on certain events during training. 

For example, the `DetectronCheckpointHook` tells the trainer to read and write model checkpoints in the [Detectron](https://github.com/facebookresearch/detectron2) format. This will read in a checkpoint at the beginning of training, if one is provided, and write a checkpoint after each epoch. In this case, our backbone weights will be the starting checkpoint.

The `AMP_Hook` applies automatic mixed precision to the model, based on the optimization level we set.

The `IterTimerHook` and `TextLoggerHook` record information about training step time and loss values at each iteration, and format the results to be easy to read in AWS CloudWatch.

The `COCOEvaluation` run evaluation either after each epoch, or at the end of training.

We can supply these hooks to the configuration as a list of strings.

In [11]:
cfg.HOOKS=["DetectronCheckpointHook",
           "AMP_Hook",
           "IterTimerHook",
           "TextLoggerHook",
           "COCOEvaluation"]

***
Let's save this configuration file as a yaml, so we have a record of how we created this training.

In [12]:
import yaml
from contextlib import redirect_stdout
from datetime import datetime

In [13]:
local_config_file = f"configs/local-config-studio.yaml"
with open(local_config_file, 'w') as outfile:
    with redirect_stdout(outfile): print(cfg.dump())

***
Here's how you can load this saved yaml file and map it to the configuration for future training sessions.

In [14]:
cfg.merge_from_file(local_config_file)

***
Now let's actually run some training. Since we expect users will want to modify the training for their own cases, we'll use some of the training tools more directly than normal.

explain why we want to do this interactively all in train.py

In [15]:
import torch
from sagemakercv.detection.detector import build_detection_model # takes cfg builds model
from sagemakercv.training import make_optimizer, make_lr_scheduler # takes cfg builds opt sch
from sagemakercv.data import make_data_loader, Prefetcher # takes cfg builds data loader
from sagemakercv.utils.runner import build_hooks, Runner # model trainer
from sagemakercv.training.trainers import train_step # actual train step
from sagemakercv.utils.runner.hooks.checkpoint import DetectronCheckpointHook # need to run this hook first, explain why

***
Create our dataloader

`num_iterations` is the number of expected steps per epoch. It's the size of the dataset divided by the global batch size. It's used so that we can keep track of when we've reached the end of an epoch. For this small local training, we'll actually be training for fewer steps than a full epoch.

In [16]:
train_coco_loader, num_iterations = make_data_loader(cfg, is_distributed=False) #local traiining explain num_iterations

NOTE! Installing ujson may make loading annotations faster.
loading annotations into memory...
Done (t=18.26s)
creating index...
index created!
shapes=None
passthrough=False


***
The prefetcher boosts performance by asynchronously grabbing the next training element, and sending it to the GPU before it's needed.

In [17]:
device = torch.device(cfg.MODEL.DEVICE) # tell torch to use the GPU not the CPU as in default config
train_iterator = Prefetcher(iter(train_coco_loader), device)

[2021-11-13 00:24:09.508 dlc-pytorch-1-9-ml-p3-2xlarge-06e8ecfc47b69e8ad61d081f6782:1066 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-13 00:24:09.552 dlc-pytorch-1-9-ml-p3-2xlarge-06e8ecfc47b69e8ad61d081f6782:1066 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


***
Build the model, optimizer, and scheduler.

In [18]:
model = build_detection_model(cfg)
model.to(device) #send to GPU
optimizer = make_optimizer(cfg, model)
scheduler = make_lr_scheduler(cfg, optimizer)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/conv.py:1083: UserWarning: _ConvTransposeMixin is a deprecated internal class. Please consider using public APIs.
  warnings.warn(


***
Build the hooks we set in the config.

In [19]:
hooks = build_hooks(cfg)

***
Now we're to build a runner, which is a training tool designed for specifically for SageMaker. It manages your training steps, hooks, and logs, making it easier to track models on CloudWatch as well as managing sending data back and forth from S3. To build the runner, we need to define a `train_step`, which is just the standard pytorch training step. This is similar to Keras in TensorFlow but more customizable. For example, a very basic train_step would be something like:

```
def train_step(inputs, model, optimizer, scheduler):
    optimizer.zero_grad()
    loss = model(inputs)
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss
```

In this case, the `train_step` can be found in `sagemakercv/training/trainers.py` and includes all steps needed for Mask RCNN.

In [20]:
runner = Runner(model, train_step, cfg, device, optimizer, scheduler)

***
Register hooks with the runner so it knows when to trigger them. [explain detectron]

In [21]:
for hook in hooks:
    runner.register_hook(hook, priority='HIGHEST' if isinstance(hook, DetectronCheckpointHook) else 'NORMAL')

***
And finally we're ready to run training. This will print a lot of info as it's setting up the model. When training starts, you'll sometimes see a few `Gradient Overflow` warnings. This is fine, it's just the mixed precision adjusting the loss scaling. After about a minute you should start seeing loss values in step increments of 50. At 2500 steps on a G4dn instane, this takes about an hour. You can speed this up by reducing the number of training steps, or using a P3.2xlarge instance. You should get a [MaP score](https://towardsdatascience.com/map-mean-average-precision-might-confuse-you-5956f1bfa9e2) of around .1 for both box and mask after 2500 steps.

In [22]:
runner.run(train_iterator, num_iterations)

Start running, work_dir: /root/smcv-pytorch-tutorial/model-output


INFO:sagemakercv.utils.runner.runner:Start running, work_dir: /root/smcv-pytorch-tutorial/model-output


max: 1 epochs


INFO:sagemakercv.utils.runner.runner:max: 1 epochs
INFO:sagemakercv.utils.checkpoint:Loading checkpoint from /root/smcv-pytorch-tutorial/model-output/model_0000100.pth
INFO:sagemakercv.utils.model_serialization:graphable.anchor_generator.cell_anchors.0                  loaded from graphable.anchor_generator.cell_anchors.0                  of shape (3, 4)
INFO:sagemakercv.utils.model_serialization:graphable.anchor_generator.cell_anchors.1                  loaded from graphable.anchor_generator.cell_anchors.1                  of shape (3, 4)
INFO:sagemakercv.utils.model_serialization:graphable.anchor_generator.cell_anchors.2                  loaded from graphable.anchor_generator.cell_anchors.2                  of shape (3, 4)
INFO:sagemakercv.utils.model_serialization:graphable.anchor_generator.cell_anchors.3                  loaded from graphable.anchor_generator.cell_anchors.3                  of shape (3, 4)
INFO:sagemakercv.utils.model_serialization:graphable.anchor_generator.cell_a

FP16_Optimizer processing param group 0:
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([512, 1, 1, 256])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 1, 1, 256])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 3, 3, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([512, 1, 1, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 1, 1, 512])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 3, 3, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([512, 1, 1, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 1, 1, 512])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 3, 3, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([512, 1, 1, 128])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 1, 1, 512])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([128, 3, 3, 128])

...

INFO:sagemakercv.utils.runner.runner:End time: 2021-11-13 00:24:17.688505


Elapsed time: 0:15:18.308144


INFO:sagemakercv.utils.runner.runner:Elapsed time: 0:15:18.308144


Evaluating predictions
creating index...
Loading and preparing results...
resFile is /root/smcv-pytorch-tutorial/model-output/inference/evaluation_dataset_0/bbox.json
DONE (t=3.99s)
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.85s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.09681
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.21110
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.07665
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.05009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.11171
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.11917
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.12888
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.21841
 Average Recall     (AR) @[ IoU

INFO:sagemakercv.utils.runner.runner:{1: (0.09680524081570316, 0.1013806033995972)}


***

Okay, so we printed a lot of numbers. Before moving on, let's first visualize what our model learned so we can make sure it's working how we expect.

SageMakerCV includes some simple visualization tools.

In [23]:
import os

import torch
import gc
from sagemakercv.data.datasets.evaluation.coco.coco_labels import coco_categories
from sagemakercv.utils.visualize import Visualizer

In [24]:
viz = Visualizer(model, cfg, temp_dir=cfg.OUTPUT_DIR, categories=coco_categories)

The visualizer can take a local file path, S3 location, or web address for an image. It will run the image through the model you just trained, and output it's predictions for objects it finds.

You can use any images you want, either from the Coco data we downloaded earlier, or your own. For example, [pixalbay](https://pixabay.com/) has lots of free use images. Just copy the image address into the `image_src` below.

In [25]:
image_src = 'https://cdn.pixabay.com/photo/2021/07/29/14/48/new-york-6507350_1280.jpg'
#'https://cdn.pixabay.com/photo/2020/05/12/11/39/cat-5162540__480.jpg'
viz(image_src, threshold=0.9) # Threshold is the minimum probability to display. Usually set around .75 or .9 so we don't get a bunch of spots where the model says there's a 2% chance of hot dogs.

RuntimeError: no valid convolution algorithms available in CuDNN

***

Note that running interactively in a notebook can build of GPU memory pressure over time. If you run inference a lot of times, you might eventually get a cuda out of memory error. If this happens, just run the garbage collection in the cell below.

In [26]:
torch.cuda.empty_cache()
gc.collect()

0

***

Before moving on, let's delete our model and related training tools so we don't keep too much in memory. The model you just trained is saved in your `cfg.OUTPUT_DIR` directory, so you can reload it later. We'll cover how to import a saved model in the final section.

In [27]:
del model, optimizer, scheduler, runner

torch.cuda.empty_cache()
gc.collect()

0

***

### Distributed Training

Great! You've managed to train a model (albeit for just a few thousand steps) on your studio instance. For many users this will be all they need. If you only intend to deal with small amounts of data (<10G) that can be trained on a single GPU, you can do everything you need with what we've run so far.

However, most practical applications of Mask RCNN require training on huge datasets across many GPUs or nodes. For that, we'll need to launch a SageMaker training job. Here we can train a model on as many as 512 [A100 GPUs](https://www.nvidia.com/en-us/data-center/a100/). We won't go quite that far. Instead, let's try training on 32 A100 GPUs across 4 P4d nodes. We'll also cover how to modify the configuration for different GPU types and counts.

The section below is also replicated in the `SageMaker.ipynb` notebook for future training once all the above setup is complete.

Before we get started, a few notes about how SageMaker training instances work. SageMaker takes care of a lot of setup for you, but it's important to understand a little of what's happening under the hood so you can customize training to your own needs. 

First we're going to look at a toy estimator to explain what's happening:

```
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                entry_point='train.py', 
                source_dir='.', 
                py_version='py3',
                framework_version='1.8.1',
                role=get_execution_role(),
                instance_count=4,
                instance_type='ml.p4d.24xlarge',
                distribution=distribution,
                output_path='s3://my-bucket/my-output/',
                checkpoint_s3_uri='s3://my-bucket/my-checkpoints/',
                model_dir='s3://my-bucket/my-model/',
                hyperparameters={'config': 'my-config.yaml'},
                volume_size=500,
                code_location='s3://my-bucket/my-code/',
)
```

The estimator forms the basic configuration of your training job.

SageMaker will first launch `instance_count=4` `instance_type=ml.p4d.24xlarge` instances. The `role` is an IAM role that SageMaker will use to launch instances on your behalf. SageMaker includes a `get_execution_role` function which grabs the execution role of your current instance. Each instance will have a `volume_size=500` EBS volume attached for your model and data. On `ml.p4d.24xlarge` and `ml.p3dn.24xlarge` instance types, SageMaker will automatically set up the [Elastic Fabric Adapter](https://aws.amazon.com/hpc/efa/). EFA provides up to 400 GB/s communication between your training nodes, as well as [GPU Direct RDMA](https://aws.amazon.com/about-aws/whats-new/2020/11/efa-supports-nvidia-gpudirect-rdma/) on `ml.p4d.24xlarge`, which allows your GPUs to bypass the host and communicate directly with each other across nodes.

Next, SageMaker we copy all the contents of `source_dir='.'` first to the `code_location='s3://my-bucket/my-code/'` S3 location, then to each of your instances. One common mistake is to leave large files or data in this directory or its subdirectories. This will slow down your launch times, or can even cause the launch to hang. Make sure to keep your working data and model artifacts elsewhere on your Studio instance so you don't accidently copy them to your training instance. You should instead use `Channels` to copy data and model artifacts, which we'll cover shortly.

SageMaker will then download the training Docker image to all your instances. Which container you download is determined by `py_version='py3'` and `framework_version='1.8.1'`. You can also use your own [custom Docker image](https://aws.amazon.com/blogs/machine-learning/bringing-your-own-custom-container-image-to-amazon-sagemaker-studio-notebooks/) by specifying an ECR address with the `image_uri` option.

Before starting training, SageMaker will check your source directory for a `setup.py` file, and install if one is present. Then SageMaker will actually launch training, via `entry_point='train.py'`. Anything in `hyperparameters={'config': 'my-config.yaml'}` will be passed to the training script as a command line argument (ie `python train.py --config my-config.yaml`). The distribution will determine what form of distributed training to launch. This will be covered in more detail later.

During training, anything written to `/opt/ml/checkpoints` on your training instances will be synced to `checkpoint_s3_uri='s3://my-bucket/my-checkpoints/'` at the same time. This can be useful for checkpointing a model you might want to restart later, or for writting Tensorboard logs to monitor your training.

When training complets, you can write your model artifacats to `/opt/ml/model` and it will save to `model_dir='s3://my-bucket/my-model/'`. Another option is to also write model artifacts to your checkpoints file.

Training logs, and any failure messages will to written to `/opt/ml/output` and saved to `output_path='s3://my-bucket/my-output/'`.

In [28]:
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

First we need to set some names. You want `AWS_DEFAULT_REGION` to be the same region as the S3 bucket your created earlier, to ensure your training jobs are reading from nearby S3 buckets.

Next, set a `user_id`. This is just for naming your training job so it's easier to find later. This can be anything you like. We also get the current date and time to make organizing training jobs a little easier.

In [29]:
# explain region. Don't launch a training job in VA with S3  bucket in OR
os.environ['AWS_DEFAULT_REGION'] = region # This is the region we set at the beginning, when creating the S3 bucket for our data

# this is all for naming
user_id="jbsnyder-smcv-tutorial" # This is used for naming your training job, and organizing your results on S3. It can be anything you like.
date_str=datetime.now().strftime("%d-%m-%Y")
time_str=datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

For instance type, we'll use an `ml.p4d.24xlarge`. We recommend this instance type for large training. It includes the latest A100 Nvidia GPUs, which can train several times faster than the previous generation. If you would rather train part way on smaller instanes, `ml.p3.2xlarge, ml.p3.8xlarge, ml.p3.16xlarge, ml.p3dn.24xlarge, ml.g4dn.12xlarge` are all good options. In particular, if you're looking for a low cost way to try a short distributed training, but aren't worried about the model fully converging, we recommend the `ml.g4dn.12xlarge` which uses 4 Nvidia T4 GPUs per node.

`s3_location` will be the base S3 storage location we used earlier for the COCO data. For `role` we get the execution role from our studio instance. For `source_dir` we use the current directory. Again, make sure you haven't accidently written any large files to this directory.

In [30]:
# specify training type, s3 src and nodes
instance_type="ml.p4d.24xlarge" # This can be any of 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'ml.p3.16xlarge', 'ml.p3.8xlarge', 'ml.p3.2xlarge', 'ml.g4dn.12xlarge'
nodes=4
s3_location=os.path.join("s3://", S3_BUCKET, S3_DIR)
role=get_execution_role() #give Sagemaker permission to launch nodes on our behalf
source_dir='.'

***
Let's modify our previous training configuration for multinode. We don't need to change much. We'll increase the batch size since we have more and large GPUs. For A100 GPUs a batch size of 12 per GPU works well. For V100 and T4 GPUs, a batch size of 6 per GPU is recommended. Make sure to lower the learning rate and increase your number of training steps if you decrease the batch size. For example, if you want to train on 2 `ml.g4dn.12xlarge` instances, you'll have 8 T4 GPUs. A batch size of `cfg.SOLVER.IMS_PER_BATCH=48`, with inference batch size of `cfg.TEST.IMS_PER_BATCH=32`, learning rate of `cfg.SOLVER.BASE_LR=0.006`, and training steps of `cfg.SOLVER.MAX_ITER=25000` is probably about right. 

The configuration below has been tested to converge to better than MLPerf accuracy.

In [31]:
cfg.SOLVER.OPTIMIZER="NovoGrad" # Type of optimizer, NovoGrad, Adam, SGD, Lamb
cfg.SOLVER.BASE_LR=0.042 # Learning rate after warmup
cfg.SOLVER.BETA1=0.9 # Beta value for Novograd, Adam, and Lamb
cfg.SOLVER.BETA2=0.3 # Beta value for Novograd, Adam, and Lamb
cfg.SOLVER.ALPHA=.001 # Alpha for final value of cosine decay
cfg.SOLVER.LR_SCHEDULE="COSINE" # Decay type, COSINE or MULTISTEP
cfg.SOLVER.IMS_PER_BATCH=384 # Global training batch size, must be a multiple of the number of GPUs
cfg.SOLVER.WEIGHT_DECAY=0.001 # Training weight decay applied as decoupled weight decay on optimizer
cfg.SOLVER.MAX_ITER=5000 # Total number of training steps
cfg.SOLVER.WARMUP_FACTOR=.01 # Starting learning rate as a multiple of the BASE_LR
cfg.SOLVER.WARMUP_ITERS=625 # Number of warmup steps to reach BASE_LR
cfg.SOLVER.GRADIENT_CLIPPING=0.0 # Gradient clipping norm, leave as 0.0 to disable gradient clipping
cfg.OPT_LEVEL="O4" # Mixed precision optimization level
cfg.TEST.IMS_PER_BATCH=128 # Evaluation batch size, must be a multiple of the number of GPUs
cfg.TEST.PER_EPOCH_EVAL=False # Eval after every epoch or only at the end of training

***
Earlier we mentioned the `distrbution` strategy in SageMaker. Distributed training can be either multi GPU single node (ie training on 8 GPU in a single ml.p4d.24xlarge) or mutli GPU multi node (ie training on 32 GPUs across 4 ml.p4d.24xlarges). For PyTorch you can use either [PyTorch's built in DDP](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) or [SageMaker Distributed Data Parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) (SMDDP). For single node multi GPU we recommend PyTorch DDP, while for multinode we recommend SMDDP. SMDDP is built to fully utilize AWS network topology, and EFA, providing a speed boost on multinode.

To enable SMDDP, set `distribution = { "smdistributed": { "dataparallel": { "enabled": True } } }`. SageMakerCV already has SMDDP integrated. To implement SMDDP for your own models, follow [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-intro.html). SMDDP will launch training from the first node in your cluster using [MPI](https://www.open-mpi.org/).

For PyTorch DDP we'll actually disable SageMaker's distribution tool, and set it up manually. When you disable distribution, SageMaker will simply launch the same `entry_point` script on each node. The setup below is to run PyTorch DDP for multi GPU on a single node, but you can use the same setup to run PyTorch DDP multinode as well. Manually setting up your own distribution can be useful when you want to do complex custom distribution strategies. For example, you can use the `SM_CURRENT_HOST` environment variable on each node to set node specific parameters. PyTorch DDP requires running the `torch.distributed.launch` on each training node. The `launch_ddp.py` script grabs the environment variables, then launches `train.py` with `torch.distributed.launch`.

In [32]:
if nodes>1 and instance_type in ['ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'ml.p3.16xlarge']:
    distribution = { "smdistributed": { "dataparallel": { "enabled": True } } } 
    entry_point = "train.py"
else:
    distribution = None
    entry_point = "launch_ddp.py"

***
We'll set a job name based on the user name and time. We'll then set output directories on S3 using the date and job name.

For this training, we'll use the same S3 location for all 3 SageMaker model outputs `/opt/ml/checkpoint`, `/opt/ml/model`, and `/opt/ml/output`.

In [33]:
job_name = f'{user_id}-{time_str}'
output_path = os.path.join(s3_location, "sagemaker-output", date_str, job_name)
code_location = os.path.join(s3_location, "sagemaker-code", date_str, job_name)

***
Next we need to add our data sources to our configuration file, but first let's talk a little more about how SageMaker gets data to your instance.

The most straightforward way to get your data is using "Channels." These are S3 locations you specify in a dictionary when you launch a training job. For example, let's say you launch a training job with:

```
channels = {'train': 's3://my-bucket/data/train/',
            'test': 's3://my-bucket/data/test/',
            'weights': 's3://my-bucket/data/weights/',
            'dave': 's3://my-bucket/data/daves_weird_data/'}

pytorch_estimator.fit(channels)
```

At the start of training, SageMaker will create a set of corresponding directories on each training node:

```
/opt/ml/input/data/train/
/opt/ml/input/data/test/
/opt/ml/input/data/weights/
/opt/ml/input/data/dave/
```

SageMaker will then copy all the contents of the corresponding S3 locations to these directories, which you can then access in training.

One downside of setting up channels like this is that it requires all the data to be downloaded to your instance at the start of of training, which can delay the training launch if you're dealing with a large dataset.

We have two ways to speed up launch. The first is [Fast File Mode](https://aws.amazon.com/about-aws/whats-new/2021/10/amazon-sagemaker-fast-file-mode/) which downloads data from S3 as it's requested by the training model, speeding up your launch time. You can use fast file mode by sepcifying `TrainingInputMode='FastFile'` in your SageMaker estimator configuration. 

If you're dealing with really huge data, on the order of several terabytes, you might not want to keep if on the instance at all, and just stream it directly from S3 into your model. The [PyTorch S3 plugin](https://aws.amazon.com/blogs/machine-learning/announcing-the-amazon-s3-plugin-for-pytorch/) provides just this capability. To use the PyTorch S3 plugin, you need to build a PyTorch dataset using the S3 plugin base class. S3 plugin support is already built into SageMakerCV. If you want to see how it's implemented, the dataset can be found in `sagemakercv/data/datasets/coco.py`. 

In our case, we'll use a mix of channels and the S3 plugin. We'll download the smaller pieces at the start of training (the validation data, pretrained weights, and image annotations), and we'll use the S3 plugin to stream the training data, since it's large. SageMakerCV will automatically switch to the S3 plugin when you supply an S3 location in the configuration file. So all we need to do is setup our channels for the data we want to download, then give the config file the locations either on the instance or on S3.

We also want to set the output dir to `/opt/ml/checkpoints`. SageMaker will sync the contents of this directory back to S3. Let's also turn up the number of workers so the data loads fast enough for the bigger GPUs we're using. 

First, we setup our training channels.

In [34]:
channels = {'validation': os.path.join(S3_DATA_LOCATION, 'val2017'),
            'weights': S3_WEIGHTS_LOCATION,
            'annotations': os.path.join(S3_DATA_LOCATION, 'annotations')}

***
Next, in the configuration file, we need to provide the corresponding location on each training node for these channels. We need to specify locations for the validation data, train and validation annotations, and the backbone weights.

In [35]:
CHANNELS_DIR='/opt/ml/input/data/' # on node
cfg.INPUT.VAL_INPUT_DIR = os.path.join(CHANNELS_DIR, 'validation') # Corresponds to the vdalidation key in the channels
cfg.INPUT.TRAIN_ANNO_DIR = os.path.join(CHANNELS_DIR, 'annotations', 'instances_train2017.json')
cfg.INPUT.VAL_ANNO_DIR = os.path.join(CHANNELS_DIR, 'annotations', 'instances_val2017.json')
cfg.MODEL.WEIGHT=os.path.join(CHANNELS_DIR, 'weights', R50_WEIGHTS) # backbone weights file

For our training data location, we'll instead point it directly to S3 so it streams the data in.

In [36]:
cfg.INPUT.TRAIN_INPUT_DIR = os.path.join(S3_DATA_LOCATION, "train2017") # Set to S3 location so we use the S3 plugin

Set the output directory to the SageMaker's checkpoint directory. This way all the files our model writes out will be immediately copied to S3.

In [37]:
cfg.OUTPUT_DIR = '/opt/ml/checkpoints'

Turn up the number of workers for the dataloader. This is especially important when using the S3 plugin. The plugin is fast, but there's still a bit of network overhead. Having more dataloaders grabbing elements from S3 simultaneously will mean our model isn't waiting around for data to load. A good rule of thunb is to set the number of dataloaders equal to the number of vCPUs divded by the number of GPUs per instance. For example, on the `ml.p4d.24xlarge` there are 96 vCPUs and 8 GPUs, so we'll use 12 workers.

In [38]:
cfg.DATALOADER.NUM_WORKERS=12 

In [39]:
cfg.HOOKS=["DetectronCheckpointHook",
           "AMP_Hook",
           "IterTimerHook",
           "TextLoggerHook",
           "COCOEvaluation"]

Like we did for the local training, we'll save the configuration file so we can replicate this same training later.

In [40]:
dist_config_file = f"configs/dist-training-config.yaml"
with open(dist_config_file, 'w') as outfile:
    with redirect_stdout(outfile): print(cfg.dump())

***
And now we can launch our training job. Let's set the config file we just created as a hyperparameter for the estimator. This will get passed to our training script as a command line argument. For example, SageMaker would launch training with `python train.py --config configs/dist-training-config.yaml`

Using 4 P4d nodes, training takes about 45 minutes. This section will also print a lot of output logs. By setting `wait=False` you can avoid printing logs in the notebook. This setting will just launch the job then return, and is useful for when you want to launch several jobs at the same time. You can then montior each job from the [SageMaker Training Console](https://us-west-2.console.aws.amazon.com/sagemaker).

In [41]:
hyperparameters = {"config": dist_config_file}

estimator = PyTorch(
                entry_point=entry_point, 
                source_dir=source_dir, 
                py_version='py3',
                framework_version='1.8.1',
                role=role,
                instance_count=nodes,
                instance_type=instance_type,
                distribution=distribution,
                output_path=output_path,
                checkpoint_s3_uri=output_path,
                model_dir=output_path,
                hyperparameters=hyperparameters,
                volume_size=500,
                code_location=code_location,
                disable_profiler=True, # Reduce number of logs since we don't need profiler or debugger for this training
                debugger_hook_config=False,
)

In [42]:
estimator.fit(channels, wait=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: jbsnyder-smcv-tutorial-13-11-2021-00-41-32


2021-11-13 00:41:51 Starting - Starting the training job...
2021-11-13 00:41:54 Starting - Launching requested ML instances............
2021-11-13 00:44:13 Starting - Preparing the instances for training....................................
2021-11-13 00:49:59 Downloading - Downloading input data.........
2021-11-13 00:51:28 Training - Downloading the training image............
2021-11-13 00:53:51 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-13 00:53:49,205 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-11-13 00:53:49,287 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-13 00:53:51,506 sagemaker-training-toolkit INFO     Imported framework sa

***
### Part 5: Visualizing Results

And there you have it, a fully trained Mask RCNN model in under an hour. Now let's see how our model does on prediction by actually visualizing the output.

Our model is stored at the S3 location we gave to the training job in `output_path`. We'll need to grab the results and store them on our studio instance so we can check performance, and visualize the output.

In [ ]:
import os
from s3fs import S3FileSystem
from configs import cfg

import torch
import gc
from sagemakercv.detection.detector import build_detection_model
from sagemakercv.utils.model_serialization import load_state_dict
from sagemakercv.data.datasets.evaluation.coco.coco_labels import coco_categories
from sagemakercv.utils.visualize import Visualizer

# Turn down logging, we don't need all the info about loading the state dict
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# Reuse the local configuration file we made earlier
cfg.merge_from_file(local_config_file)

In [ ]:
s3fs = S3FileSystem()

TRAINING_OUTPUT=os.path.join(LOCAL_DATA_DIR, 'training_output') # Make a new local directory for our training results and new images
os.makedirs(TRAINING_OUTPUT, exist_ok=True)

# Grab the filename of the last checkpoint our training job wrote
s3fs.get(os.path.join(output_path, 'last_checkpoint'), os.path.join(TRAINING_OUTPUT, 'last_checkpoint'))
with open(os.path.join(TRAINING_OUTPUT, 'last_checkpoint'), 'r') as f:
    checkpoint_name=f.readline().split('/')[-1]
    
# Copy the saved weights to our local directory
s3fs.get(os.path.join(output_path, checkpoint_name), os.path.join(TRAINING_OUTPUT, checkpoint_name))

***
To run inference, we first build a blank model and then add weights from the distributed training job.

We can build a new model the same way we built one for local training. This time, instead of using the checkpointer to load the backbone weights, we'll directly load all the saved weights from the trained model.

In [ ]:
device = torch.device(cfg.MODEL.DEVICE)
model = build_detection_model(cfg)
_ = model.to(device)

# Load weights file as a dictionary
weights = torch.load(os.path.join(TRAINING_OUTPUT, checkpoint_name))['model']

# Map saved weights to our model
load_state_dict(model, weights, False)

***
Like we did with the local training, we can pass the model to the visualization tool, and give it an image either from local, S3, or a web address.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
viz = Visualizer(model, cfg, temp_dir=TRAINING_OUTPUT, categories=coco_categories)

In [ ]:
image_src = 'https://cdn.pixabay.com/photo/2021/07/29/14/48/new-york-6507350_1280.jpg'
#'https://cdn.pixabay.com/photo/2020/05/12/11/39/cat-5162540__480.jpg'
viz(image_src, threshold=0.9) #explain threshold

#### Conclusion

In this notebook, we've walked through the entire process of training Mask RCNN on SageMaker. We've implemented several of SageMaker's more advanced features, such as distributed training, EFA, and streaming data directly from S3. From here you can use the provided template datasets to train on your own data, or modify the framework with your own object detection model.

When you're done, make sure to check that all of your SageMaker training jobs have stopped by checking the [SageMaker Training Console](https://us-west-2.console.aws.amazon.com/sagemaker). Also check that you've stopped any Studio instance you have running by selecting the session monitor on the left (the circle with a square in it), and clicking the power button next to any running instances. Your files will still be saved on the Studio EBS volume.

<img src="../assets/running_instances.png" style="width: 600px">